### walkthrough of METRIC

In [1]:
import rasterio as rio
import xarray as xar
from affine import Affine
import numpy as np
import pyproj
import sys
import daymetpy
import pandas as pd
#for METRIC
from collections import deque
import time
from pathlib import Path
import numpy as np
import os
import pyTSEB.meteo_utils as met
import pyTSEB.resistances as res
import pyTSEB.MO_similarity as MO
import pyTSEB.TSEB as tseb
from cropmask import io_utils

def get_lat_lon_center(da):
    """
    Gets the center of the raster in lon, lat coordinates by reprojecting to WGS84.
    """
    center = rio.transform.xy(Affine(*da.attrs['transform']), rows=da.sizes['y']//2, cols=da.sizes['x']//2, offset='center')
    outProj =pyproj.Proj(init='epsg:4326')
    inProj = pyproj.Proj(da.attrs['crs'])
    lon,lat = pyproj.transform(inProj,outProj,center[0], center[1])
    return lon, lat
def get_lat_lon_arrs(da):
    """
    Gets the lon, lat coordinates by reprojecting to WGS84, in list of tuple form.
    """
    xs, ys = rio.transform.xy(Affine(*da.attrs['transform']), rows=np.arange(da.sizes['y']), cols=np.arange(da.sizes['x']), offset='center')
    lon_lat_arrs = []  
    outProj = pyproj.Proj(init='epsg:4326')
    inProj = pyproj.Proj(da.attrs['crs'])
    lons, lats = pyproj.transform(inProj,outProj,xs, ys)
    return list(zip(lons, lats))

In [2]:
# os.chdir("/mnt/cropmaskperm/")

In [3]:
# test vars
Tr_K = xar.open_rasterio("test_metric/LT05_CU_012006_20020825_20190517_C01_V01_ST/LT05_CU_012006_20020825_20190517_C01_V01_ST.tif", chunks = {'x':500, 'y':500})\
    .squeeze()
L_dn = xar.open_rasterio("test_metric/LT05_CU_012006_20020825_20190517_C01_V01_ST/LT05_CU_012006_20020825_20190517_C01_V01_DRAD.tif", chunks = {'x':500, 'y':500})\
    .squeeze()
emis = xar.open_rasterio("test_metric/LT05_CU_012006_20020825_20190517_C01_V01_ST/LT05_CU_012006_20020825_20190517_C01_V01_EMIS.tif", chunks = {'x':500, 'y':500})\
    .squeeze()

band_paths = list(Path("test_metric/LT05_CU_012006_20020825_20190517_C01_V01_SR/").glob("*B*.tif")) # grab, sort and read in bands as xarr
band_paths = sorted(band_paths)
SR = io_utils.read_bands_lsr(band_paths)
SR = SR.transpose('y', 'x', 'band')

center = get_lat_lon_center(Tr_K)

In [4]:
coords = get_lat_lon_arrs(Tr_K)
coord_arr = np.array(coords, dtype=np.dtype("float,float"))

In [5]:
# need to test if numpy masked arrays or xarray masking works better. using xarr for now
# np.ma.masked_where(L_dn==-9999, L_dn)

emis = emis.where(L_dn!=-9999)
emis = emis * .0001

L_dn = L_dn.where(L_dn!=-9999)
L_dn = L_dn * .001

Tr_K = Tr_K.where(Tr_K!=-9999)
Tr_K = Tr_K * .1

In [6]:
aoi_met = daymetpy.daymet_timeseries(lon=center[0], lat=center[1], start_year=2002, end_year=2002) # used to estimate mock params for aoi

In [7]:
import pvlib
import datetime
from timezonefinder import TimezoneFinder

time = pd.to_datetime(datetime.datetime(2002, 8, 3, 10, 4))
time = pd.DatetimeIndex([time])
tf = TimezoneFinder(in_memory=True)
timezone = tf.certain_timezone_at(lng=center[0], lat=center[1]) 
time = time.tz_localize(timezone)
solar_df = pvlib.solarposition.get_solarposition(time,center[1], center[0]) # can be made more accurate with temp and pressure from daymet

# hot and cold pixel search

In [8]:
ltype = "Landsat5"

if ltype in ['Landsat4', 'Landsat5', 'Landsat7']:
    thermal_band = '6'
    # "wb" coefficients from Trezza et al. 2008
    band_sur_dict = {
        '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '7': 6}

    wb_dict = {'1': 0.254, '2': 0.149, '3': 0.147,
                    '4': 0.311, '5': 0.103, '7': 0.036}
elif ltype in ['Landsat8']:

    
    thermal_band = '10'
    band_sur_dict = {
        '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6}
    wb_dict = {'2': 0.254, '3': 0.149, '4': 0.147,
                    '5': 0.311, '6': 0.103, '7': 0.036}
def band_dict_to_array(data_dict, band_dict):
    """
    
    Parameters
    ----------
    data_dict : dict
    band_dict: dict
    Returns
    -------
    ndarray
    """
    return np.array(
        [v for k, v in sorted(data_dict.items())
         if k in band_dict.keys()]).astype(np.float32)

# Convert dictionaries to arrays
wb = band_dict_to_array(wb_dict, band_sur_dict)

In [9]:
def mask_same_shape(function):
    def func(arr1, arr2):
        # tried basing on single nodata vlaue but ARD has negative val artifacts over water
        arr1, arr2= arr1.where(arr2 > 0), arr2.where(arr1 > 0)
        return function(arr1, arr2)
    return func

@mask_same_shape
def savi(nir, r):
    """
    In Landsat 4-7, SAVI = ((Band 4 – Band 3) / (Band 4 + Band 3 + 0.5)) * (1.5).

    In Landsat 8, SAVI = ((Band 5 – Band 4) / (Band 5 + Band 4 + 0.5)) * (1.5).
        
    https://www.usgs.gov/land-resources/nli/landsat/landsat-soil-adjusted-vegetation-index
    """
    
    return ((nir - r) / (nir + r + 0.5)) * (1.5)

@mask_same_shape
def ndvi(nir, r):
    """
    In Landsat 4-7, SAVI = ((Band 4 – Band 3) / (Band 4 + Band 3)).

    In Landsat 8, SAVI = ((Band 5 – Band 4) / (Band 5 + Band 4 )).
    """
    
    return (nir - r) / (nir + r )

def reflectance_to_albedo(refl_sur, wb):
    """Tasumi at-surface albedo
    Parameters
    ----------
    refl_sur : ndarray
    wb :
    Returns
    -------
    ndarray
    References
    ----------
    .. [1] Tasumi, M., Allen, R., and Trezza, R. (2008). At-surface reflectance
       and albedo from satellite for operational calculation of land surface
       energy balance. Journal of Hydrologic Engineering 13(2):51-63.
       https://doi.org/10.1061/(ASCE)1084-0699(2008)13:2(51)
    """
    return np.sum(refl_sur * wb, axis=2)

def savi_lai_func(savi):
    """Compute leaf area index (LAI) from SAVI
    Parameters
    ----------
    savi : array_like
        Soil adjusted vegetation index. 
        
    Returns
    -------
    ndarray
    References
    ----------
    .. [1] Allen, R., Tasumi, M., & Trezza, R. (2007). Satellite-Based Energy
       Balance for Mapping Evapotranspiration with Internalized Calibration
       (METRIC)-Model. Journal of Irrigation and Drainage Engineering, 133(4).
       https://doi.org/10.1061/(ASCE)0733-9437(2007)133:4(380)
    """
    return np.clip((11. * np.power(savi, 3)), 0, 6)


def ndvi_lai_func(ndvi):
    """Compute leaf area index (LAI) from NDVI
    Parameters
    ----------
    ndvi : array_like
        Normalized difference vegetation index.
    Returns
    -------
    ndarray
    References
    ----------
    .. [1] Trezza and Allen 2014?
    """
    return np.clip((7. * np.power(ndvi, 3)), 0, 6)

In [10]:
# from nlcd metric file on DRI pymetric
zom_nlcd_remap = {
  "11": "0.0005", # open water
  "12": "0.005", # perennial ice/snow
  "21": "0.05", # developed, open space
  "22": "0.08", # developed, low intensity
  "23": "0.1", # developed, medium
  "24": "0.2", # developed, high
  "31": "0.1", # barren land (rock sand clay)
  "32": "0.005", 
  "41": "perrier", # deciduous forest
  "42": "perrier", # evergreen forest
  "43": "perrier", # mixed forest
  "51": "0.2", # dwarf scrub
  "52": "0.2", # shrb/scrub
  "71": "0.05", # grassland/herbaceaous
  "72": "0.03", # sedge/Herbaceaous
  "81": "LAI", 
  "82": "LAI", # pasture/hay
  "90": "0.4", # cultivated crops
  "94": "0.2", # woody wetlands
  "95": "0.01" # emergent herbaceaous wetlands
 }
# https://www.mrlc.gov/data/legends/national-land-cover-database-2011-nlcd2011-legend

In [11]:
# better way to calc zero plane displacement for non crops https://github.com/hectornieto/pyTSEB/blob/b940cfb00512acb3c1a2270928600b4f5b3191cc/pyTSEB/resistances.py
def calc_d_0(h_C):
    ''' Zero-plane displacement height
    Calculates the zero-plane displacement height based on a
    fixed ratio of canopy height.
    Parameters
    ----------
    h_C : float
        canopy height (m).
    Returns
    -------
    d_0 : float
        zero-plane displacement height (m).'''

    d_0 = h_C * 0.65

    return np.asarray(d_0)


def zom_func(lai, landuse, zom_remap):
    """Generate Zom (roughness) values based on the landuse type
    Parameters
    ----------
    lai : ndarray
        Leaf area index.
    landuse : ndarray
        Landuse.
    zom_remap : dict
        Mapping of landuse types to zom values in JSON format with key/value
        both string type (i.e. "11" : "0.005").
    Returns
    -------
    ndarray
    References
    ----------
    .. [1] Allen, R., Tasumi, M., & Trezza, R. (2007). Satellite-Based Energy
       Balance for Mapping Evapotranspiration with Internalized Calibration
       (METRIC)-Model. Journal of Irrigation and Drainage Engineering, 133(4).
       https://doi.org/10.1061/(ASCE)0733-9437(2007)133:4(380)
    """
    zom = np.full(lai.shape, 0.005, dtype=np.float32)

    for lu_code in np.unique(landuse):
        # What should be the default zom value?
        # Convert the landuse array values to strings for now.
        try:
            lu_value = zom_remap[str(lu_code)]
        except:
            lu_value = 'lai'

        if lu_value.lower() == 'perrier':
            zom[landuse == lu_code] = perrier_zom_func(lai[landuse == lu_code])
        elif lu_value.lower() == 'lai':
            zom[landuse == lu_code] = np.maximum(
                lai[landuse == lu_code] * 0.018, 0.005)
        else:
            zom[landuse == lu_code] = float(lu_value)

    zom[np.isnan(lai)] = np.nan
    return zom


def perrier_zom_func(lai):
    """Perrier Zom
    Parameters
    ----------
    lai : ndarray
        Leaf area index.
    Returns
    -------
    ndarray
    Notes
    -----
    Minimum zom is 0.005 m equal to bare soil. Dec 28 09, JK
    The use of the function is applicable for tall vegetation (forests).
    The canopy distribution coefficient, a, is assumed to be a=0.6,
        i.e. slightly top heavy canopy.
    The vegetation height is estimated as h=2.5LAI (LAImax=6 -> 2.5*6=15 m),
        compared to h=0.15LAI for agriculture crops.
    References
    ----------
    .. [1] Perrier, A. (1982). Land surface processes: Vegetation.
       In Land Surface Processes in Atmospheric General Circulation Models;
       Eagelson, P.S., Ed.; Cambridge University Press: Cambridge, UK;
       pp. 395-448.
    .. [2] Allen, R., Irmak, A., Trezza, R., Hendrickx, J., Bastiaanssen, W.,
       & Kjaersgaard, J. (2011). Satellite-based ET estimation in agriculture
       using SEBAL and METRIC. Hydrologic Processes, 25, 4011-4027.
       https://doi.org/10.1002/hyp.8408
    .. [3] Santos (2012)
    """
    perrier = -1.2 * lai
    perrier /= 2.
    np.exp(perrier, out=perrier)
    perrier = ((1 - perrier) * perrier) * (2.5 * lai)
    return np.maximum(perrier, 0.005, dtype=np.float32)

need to run with more memory, clip nlcd to ard extent and test METRIC

In [12]:
# import cropmask.label_prep as lp
# from rasterio.warp import reproject
# with rio.open("NLCD_tif/NLCD_2001_Land_Cover_L48_20190424.tif") as nlcd:
#     with rio.open("test_metric/LT05_CU_012006_20020825_20190517_C01_V01_ST/LT05_CU_012006_20020825_20190517_C01_V01_ST.tif") as ard:
#         ard_poly = lp.rio_bbox_to_polygon(ard.bounds)
#         kwargs = nlcd.meta.copy()
#         kwargs.update({
#             'crs': ard.meta['crs'],
#             'transform': ard.meta['transform'],
#             'width' : nlcd.meta['width'],
#             'height' : ard.meta['height']
#         })
#         with rio.open('tmp.tif', 'w', **kwargs) as dst:
#             reproject(
#                 source=rio.band(nlcd, 1),
#                 destination=rio.band(dst, 1),
#                 src_transform=nlcd.meta['transform'],
#                 src_crs=nlcd.meta['crs'],
#                 dst_transform=ard.meta['transform'],
#                 dst_crs=ard.meta['crs'])

# METRIC model

In [13]:
# mocks
alt=0
T_A_K = 25.0+273.15
u = 2 #m/s
ea = 20 # mb, mock value under the saturation vapor pressure at 25 Celsius
p = 1013 # mb
S_dn = 500 # flux density, guestimate based on center of image and rough day of year daymet data
L_dn = L_dn
emis = emis
z_0M = .018 # surface roughness length for momentum transport, see https://reader.elsevier.com/reader/sd/pii/S0022169498002534?token=B4ADFCE769E6A06E951B6DDF5DBBF54EB29B4C76713AA74F1322C942C1B9C560F655D77CD6EDA4CA9D2E71C711AD7167
h_C = 2 # mock canopy height of 2 meters
d_0 = calc_d_0(np.ones(Tr_K.shape)*h_C)
z_u = 2 # height of windspeed measurement
z_T = 2 # height of air temperature measurement
LE_hot=0
use_METRIC_resistance=True
calcG_params=[[0], 0.15]
UseL=False
UseDEM=False


end member search

In [14]:
from pyMETRIC import endmember_search

VI = ndvi(SR.sel(band=4), SR.sel(band=3))

VI_MAX = 0.95

albedo = reflectance_to_albedo(SR.where(SR>0), wb)

# Reduce potential ET based on vegetation density based on Allen et al. 2013
ET_r_f_cold = xar.ones_like(Tr_K) * 1.05
ET_bare_soil = xar.zeros_like(Tr_K)
ET_r_f_cold = xar.where(VI < VI_MAX, 1.05/VI_MAX * VI, ET_r_f_cold) # Eq. 4 [Allen 2013]

ET_r_f_hot = VI * ET_r_f_cold + (1.0 - VI) * ET_bare_soil # Eq. 5 [Allen 2013]

# Compute normalized temperatures by adiabatic correction
gamma_w = met.calc_lapse_rate_moist(Tr_K,
                                    ea,
                                    p)

Tr_datum = Tr_K + gamma_w * alt
Ta_datum = T_A_K + gamma_w * alt

check for infinite values

In [15]:
values = [np.inf,np.nan,0]
print(np.in1d(values, VI))

[False False  True]


In [16]:
# Compute spatial homogeneity metrics
cv_ndvi, _, _ = endmember_search.moving_cv_filter(VI, (11, 11))
cv_lst, _, std_lst = endmember_search.moving_cv_filter(Tr_datum, (11, 11))
cv_albedo,_, _ = endmember_search.moving_cv_filter(albedo, (11, 11))

/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:343: RuntimeWarning: divide by zero encountered in true_divide
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:343: RuntimeWarning: invalid value encountered in true_divide


In [17]:
cold_pixel, hot_pixel = endmember_search.esa(VI,
                                Tr_datum,
                                cv_ndvi,
                                std_lst,
                                cv_albedo)
            

Filtering pixels by homgeneity


/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:180: RuntimeWarning: invalid value encountered in less_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:181: RuntimeWarning: invalid value encountered in less_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:182: RuntimeWarning: invalid value encountered in less


Found 6023919 homogeneous pixels
Setting LST boundaries 255.70000000000002 - 323.90000000000003
Setting VI boundaries -0.9193235375658442 - 0.999100404817832
Setting LST boundaries 256.40000000000003 - 323.6
Setting VI boundaries -0.9054119941491955 - 0.999100404817832
Setting LST boundaries 256.90000000000003 - 323.3
Setting VI boundaries -0.8886444406837029 - 0.999100404817832
Setting LST boundaries 256.90000000000003 - 323.0
Setting VI boundaries -0.8540145985401459 - 0.999100404817832
Setting LST boundaries 256.90000000000003 - 322.70000000000005
Setting VI boundaries -0.8438277864847423 - 0.999100404817832
Setting LST boundaries 256.90000000000003 - 322.40000000000003
Setting VI boundaries -0.8304997424008244 - 0.999100404817832
Setting LST boundaries 256.90000000000003 - 322.1
Setting VI boundaries -0.8157058556513844 - 0.999100404817832
Setting LST boundaries 256.90000000000003 - 322.1
Setting VI boundaries -0.7746575342465754 - 0.999100404817832
Setting LST boundaries 256.90000

/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:194: RuntimeWarning: invalid value encountered in greater_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:195: RuntimeWarning: invalid value encountered in less_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:196: RuntimeWarning: invalid value encountered in greater_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:197: RuntimeWarning: invalid value encountered in less_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:303: RuntimeWarning: invalid value encountered in greater_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:30

Found 3455 candidate cold pixels
Iterative search of candidate hot pixels
Searching hot pixels from the 1 % maximum LST and 1 % minimum VI


/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:322: RuntimeWarning: invalid value encountered in less_equal
/home/rave/miniconda3/envs/METRIC/lib/python3.7/site-packages/pyMETRIC-1.0-py3.7.egg/pyMETRIC/endmember_search.py:323: RuntimeWarning: invalid value encountered in greater_equal


Found 42 candidate hot pixels
Ranking candidate anchor pixels
Cold  pixel found with 297.0 K and 0.7064062051134732 VI
Hot  pixel found with 319.0 K and 0.06318614296169797 VI


Scale factors for temperature product variables documented in: https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1330-LandsatSurfaceTemperature_ProductGuide-v2.pdf

In [18]:
from pyMETRIC.METRIC import pet_asce

LE_potential = pet_asce(Ta_datum,
                              u,
                              ea,
                              p,
                              S_dn,
                              z_u,
                              z_T,
                              f_cd=1,
                              reference=True)

In [ ]:
from pyMETRIC.METRIC import METRIC

        
flag, R_nl1, LE1, H1, G1, R_A1, u_friction, L, n_iterations = \
         METRIC(Tr_K,
                T_A_K,
                u,
                ea,
                p,
                S_dn,
                L_dn,
                emis,
                z_0M,
                d_0,
                z_u,
                z_T,
                cold_pixel,
                hot_pixel,
                LE_potential,
                LE_hot=0,
                use_METRIC_resistance = use_METRIC_resistance,
                calcG_params=calcG_params,
                UseDEM=UseDEM)

#### this has code specific to the SENET project, the pyMETRIC.py file. METRIC function in METRIC.py contains the model and we should focus on using that probably, but with pyMETRIC as a guide for implementing on rasters

In [ ]:
from collections import OrderedDict
from os.path import splitext, dirname, exists
from os import mkdir

import numpy as np
import ast
import gdal
from netCDF4 import Dataset

# S_* are flags to determine if output should be saved. N stands for No, P for Priamry, A for Ancillary
from pyTSEB.PyTSEB import S_N, S_P, S_A 

# this comes with all the land cover IDs for the IGBP global land cover classication. IDs will need to change for NLCD.
# also comes with func to calculate the roughness and zero plane displacement. This could be substituted for uniform veg calcualtion if we only care about crops.
from pyTSEB import resistances as res 

# calculates sun zenith angles based on time and lcoation, pressure based on altitude, downwelling longwave from air T
# and lapse rate to correct T reference and air temperature based on vapor pressure and air pressure
# not sure what the lapse rate temperature normalization is for..
from pyTSEB import meteo_utils as met


# function for bruetsart's equation for atmospheric emissivity to calc downwelling longwave,
# Esimate diffuse and direct irradiance (shortwave incoming to surface). inputs to this parsing func are 
# S_dn, incoming shortwave, solar zenith, solar constant and Sn_c canopy net shortwave radiation and Sn_S soil net shortwave radiation
from pyTSEB import net_radiation as rad
from pyMETRIC import METRIC, endmember_search
import xarray

### Questions

Wha is an adiabatic lapse rate? Why is it used to normalize temperature like so?

```python
    gamma_w = met.calc_lapse_rate_moist(in_data['T_A1'],
                                        in_data['ea'],
                                        in_data['p'])
    Tr_datum = in_data['T_R1'] + gamma_w * in_data['alt']
    Ta_datum = in_data['T_A1'] + gamma_w * in_data['alt']
```

# code graveyard

In [ ]:
def get_x_y_arrs(da):
    """
    Takes array read with open_rasterio, outputs x and y arrays of same shape.
    """
    transform = Affine.from_gdal(*da.attrs['transform'])
    nx, ny = da.sizes['x'], da.sizes['y']
    x, y = np.meshgrid(np.arange(nx)+0.5, np.arange(ny)+0.5) * transform
    return x, y

### Setup

1. Setup conda env
2. install setuptools
3. install https://github.com/hectornieto/pyPro4Sail a dependency
4. install https://github.com/hectornieto/pyMETRIC the ET model
5. install https://github.com/khufkens/daymetpy for ancillary inputs

Example reference using Daymet for ancillary inputs to estimate ET with Landsat:https://www.sciencedirect.com/science/article/pii/S0034425715302650

projection info for daymet: https://wiki.cyverse.org/wiki/display/~tyson_swetnam/Raster+Calculations

To get daymet data for a 512x512 scene and build an array it will take 235 api calls which may not be too bad.

This func will get the lat, lon tuples of a raster to be used to get daymet data: https://rasterio.readthedocs.io/en/stable/api/rasterio.transform.html

I've chosen daymet over mesonet because it is easier to acquire, but later spatially interpolated Mesonet and other data sources can be used for better accuracy. [This paper](https://www.sciencedirect.com/science/article/pii/S1161030116302076#bib0005) notes:"While, on average, simulated yields using Daymet were in closer agreement with simulations based on MWD compared with PRISM (RMSE%: 18 vs. 24%), there were still large differences in simulated yield in 20% of the site-years. And, perhaps more importantly, these differences were not predictable as they were not associated with any spatial pattern in weather, topography, or weather network. While GWD might be useful for applications that only require temperature, such as crop stage prediction or quantification of early frost risk, water deficit and simulated yields for specific site-years are highly uncertain and there is no way to ex-ante predict the magnitude and direction of the bias, which undermines utility of GWD for field-specific or real-time agronomic applications."

Daymet overview: https://daymet.ornl.gov/overview

Daymet has been critiqued in this 2017 paper for not homogenizing the grid to account for climate change over time in places that are far from station data. It has also been critiqued for artifacts where there is no station data (Walton and Hall, An Assessment of High-Resolution Gridded Temperature Datasets
over California)

The variables we don't have and which are most difficult to get probably are related to wind speed and vegetation structure:
u, z_0M, d_0, z_u. Maybe we get these from mesonet stations, need to look at the literature to see what folks do about this.

In [ ]:
import sys
import daymetpy
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

denver_loc = (-104.9903, 39.7392)
miami_loc = (-80.2089, 25.7753)

denver = daymetpy.daymet_timeseries(lon=denver_loc[0], lat=denver_loc[1], start_year=2012, end_year=2014)

In [ ]:
denver

In [ ]:
import pyMETRIC.METRIC as met

 Parameters
        ----------
        CHECK (landsat) Tr_K : float
            Radiometric composite temperature (Kelvin).
        CHECK (daymet convert to Kelvin, height is estimated at 2 meter) T_A_K : float
            Air temperature (Kelvin).
        (assume? mesonet stations?) u : float
            Wind speed above the canopy (m s-1).
        CHECK (daymet needs to be converted from pascals to mb) ea : float
            Water vapour pressure above the canopy (mb).
        CHECK (will use default or calc from elevation) p : float
            Atmospheric pressure (mb), use 1013 mb by default.
        CHECK (daymet, right units in W/m/-2 already, but also consider pyMETRIC suggested method from albedo...) S_n : float
            Solar irradiance (W m-2).
        (estimated from vapor pressure and air temp (to get atm emissivity)) L_dn : float
            Downwelling longwave radiation (W m-2)
        (https://modis.gsfc.nasa.gov/data/dataprod/mod11.php) emis : float
            Surface emissivity.
        CHECK (.02*uniform vegetation height=z_0m source: C&N) z_0M : float
            Aerodynamic surface roughness length for momentum transfer (m).
        CHECK (.6*uniform vegetation height=d_0 source: C&N) d_0 : float
            Zero-plane displacement height (m).
        z_u : float
            Height of measurement of windspeed (m).
        CHECK (see daymet docs, 2 meter:https://daymet.ornl.gov/overview.html) z_T : float
            Height of measurement of air temperature (m).
        cold_pixel : tuple
            pixel coordinates (row, col) for the cold endmember
        hot_pixel : tuple
            pixel coordinates (row, col) for the hot endmember
        calcG_params : list[list,float or array], optional
            Method to calculate soil heat flux,parameters.

Also these params are needed for exhaustive hot/cold pixel search, easily computed from Landsat and MODIS for albedo. Or possibly Landsat? this is what EEflux did to estiamte albedo

"Albedo – This represents total, integrated reflectance across the electromagnetic spectrum. It is calculated from the six primary short-wave bands and is useful for energy balance work. Based on Tasumi et al., 2008, ASCE J. Hydrologic Engineering."

    vi_array : numpy array
        Vegetation Index array (-)
    lst_array : numpy array
        Land Surface Temperature array (Kelvin)
    cv_ndvi : numpy array
        Coefficient of variation of Vegetation Index as homogeneity measurement
        from neighboring pixels
    std_lst : numpy array
        Standard deviation of LST as homogeneity measurement
        from neighboring pixels
    cv_albedo : numpy array
        Coefficient of variation of albdeo as homogeneity measurement
        from neighboring pixels

[Good overview of METRIC](https://onlinelibrary.wiley.com/doi/full/10.1111/jawr.12056)

In [ ]:
import pyMETRIC.METRIC as METRIC 
output=METRIC.METRIC(Tr_K, T_A_K, u, ea, p, Sn, L_dn, emis, z_0M, d_0, z_u, z_T, cold_pixel, hot_pixel, LE_cold)

In [ ]:
help(met)